# Advanced Nuclear Fusion Analysis

This notebook demonstrates advanced analysis techniques for nuclear fusion data including time series analysis, optimization, and physics-based constraints.

## Contents
1. Time Series Analysis
2. Parameter Optimization
3. Physics-based Validation
4. Sensitivity Analysis
5. Uncertainty Quantification
6. Multi-objective Optimization
7. Scenario Analysis
8. Real-time Monitoring Simulation

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize, stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid

# Import fusion analyzer modules
from src.data.generator import FusionDataGenerator
from src.data.processor import FusionDataProcessor
from src.models.fusion_predictor import FusionPredictor
from src.models.anomaly_detector import FusionAnomalyDetector
from src.utils.evaluator import FusionModelEvaluator
from src.visualization.plotter import FusionPlotter
from src.utils.config_manager import get_global_config

# Helper function for categorizing Q factor
def categorize_q_factor(q):
    """Categorize Q factor performance level."""
    if q < 0.1:
        return "Very Poor"
    elif q < 0.5:
        return "Poor"
    elif q < 1.0:
        return "Sub-critical"
    elif q < 5.0:
        return "Break-even"
    elif q < 10.0:
        return "Ignition"
    else:
        return "High Performance"

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Configuration loaded: {get_global_config().config_name}")

## 1. Time Series Analysis

In [ ]:
# Generate time series data for fusion reactor operation
generator = FusionDataGenerator(random_state=42)

# Simulate a fusion shot with varying parameters
base_params = {
    'magnetic_field': 5.3,
    'plasma_current': 15.0,
    'electron_density': 1.0e20,
    'ion_temperature': 20.0,
    'electron_temperature': 15.0,
    'heating_power': 50.0
}

# Generate 10-second fusion shot with 0.1s resolution
time_series = generator.generate_time_series(
    n_timepoints=100,
    dt=0.1,
    base_params=base_params
)

print(f"Generated time series with {len(time_series)} time points")
print(f"Time range: {time_series['time'].min():.1f} to {time_series['time'].max():.1f} seconds")

In [ ]:
# Plot time series evolution
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

# Key parameters to plot
params_to_plot = [
    ('q_factor', 'Q Factor'),
    ('magnetic_field', 'Magnetic Field (T)'),
    ('plasma_current', 'Plasma Current (MA)'),
    ('electron_density', 'Electron Density (m⁻³)'),
    ('ion_temperature', 'Ion Temperature (keV)'),
    ('total_heating_power', 'Total Heating Power (MW)')
]

for i, (param, label) in enumerate(params_to_plot):
    if param in time_series.columns:
        axes[i].plot(time_series['time'], time_series[param], linewidth=2)
        axes[i].set_xlabel('Time (s)')
        axes[i].set_ylabel(label)
        axes[i].set_title(f'{label} Evolution')
        axes[i].grid(True, alpha=0.3)
        
        # Add breakeven line for Q factor
        if param == 'q_factor':
            axes[i].axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Breakeven')
            axes[i].axhline(y=5.0, color='orange', linestyle='--', alpha=0.7, label='Ignition')
            axes[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Analyze time series characteristics
print("Time Series Analysis:")
print("=" * 30)

# Q factor statistics
q_stats = {
    'Mean': time_series['q_factor'].mean(),
    'Max': time_series['q_factor'].max(),
    'Min': time_series['q_factor'].min(),
    'Std': time_series['q_factor'].std()
}

print("Q Factor Statistics:")
for stat, value in q_stats.items():
    print(f"  {stat}: {value:.3f}")

# Time above breakeven and ignition
breakeven_time = (time_series['q_factor'] >= 1.0).sum() * 0.1
ignition_time = (time_series['q_factor'] >= 5.0).sum() * 0.1

print(f"\nPerformance Metrics:")
print(f"  Time above breakeven: {breakeven_time:.1f}s ({breakeven_time/10*100:.1f}%)")
print(f"  Time above ignition: {ignition_time:.1f}s ({ignition_time/10*100:.1f}%)")

# Detect disruptions (sudden drops in Q factor)
q_gradient = np.gradient(time_series['q_factor'])
disruption_threshold = -0.5  # Large negative gradient
disruptions = np.where(q_gradient < disruption_threshold)[0]

print(f"\nDisruption Analysis:")
print(f"  Number of disruptions detected: {len(disruptions)}")
if len(disruptions) > 0:
    print(f"  Disruption times: {time_series.iloc[disruptions]['time'].values}")

## 2. Parameter Optimization

In [ ]:
# Set up optimization to find optimal fusion parameters
# First train a model for optimization

print("Setting up parameter optimization...")

# Generate training data
training_data = generator.generate_dataset(n_samples=5000)
processor = FusionDataProcessor(random_state=42)
processed_data = processor.preprocess_pipeline(training_data, target_column='q_factor')

# Train a fast model for optimization
predictor = FusionPredictor()
results = predictor.train_model(
    'random_forest', 
    processed_data['X_train'], 
    processed_data['y_train'],
    processed_data['X_val'], 
    processed_data['y_val']
)

print(f"Model trained with R² = {results['val_r2']:.4f}")
feature_names = processed_data['feature_names']

In [ ]:
# Define optimization objective function
def fusion_objective(params, predictor, processor, feature_names, maximize=True):
    """
    Objective function for fusion parameter optimization.
    
    Args:
        params: Array of parameter values
        predictor: Trained fusion predictor
        processor: Data processor for feature scaling
        feature_names: List of feature names
        maximize: Whether to maximize (True) or minimize (False) Q factor
    """
    try:
        # Create parameter dictionary
        param_dict = {
            'magnetic_field': params[0],
            'plasma_current': params[1], 
            'electron_density': params[2],
            'ion_temperature': params[3],
            'electron_temperature': params[4],
            'neutral_beam_power': params[5],
            'rf_heating_power': params[6]
        }
        
        # Create DataFrame
        df = pd.DataFrame([param_dict])
        
        # Process the data (feature engineering)
        engineered_df = processor.engineer_features(df)
        
        # Select and scale features to match training data
        X = np.zeros((1, len(feature_names)))
        for i, feature in enumerate(feature_names):
            if feature in engineered_df.columns:
                X[0, i] = engineered_df[feature].iloc[0]
        
        # Make prediction
        q_pred = predictor.predict(X, 'random_forest')[0]
        
        # Return negative for minimization (scipy minimizes)
        return -q_pred if maximize else q_pred
        
    except Exception as e:
        # Return poor value if calculation fails
        return 1000 if maximize else -1000

# Define parameter bounds (realistic ranges)
bounds = [
    (3.0, 8.0),    # magnetic_field
    (10.0, 25.0),  # plasma_current  
    (5e19, 2e20),  # electron_density
    (10.0, 40.0),  # ion_temperature
    (8.0, 35.0),   # electron_temperature
    (20.0, 100.0), # neutral_beam_power
    (10.0, 60.0)   # rf_heating_power
]

# Initial guess (reasonable starting point)
x0 = [5.3, 15.0, 1.0e20, 20.0, 15.0, 50.0, 30.0]

print("Starting parameter optimization...")
print(f"Initial guess Q factor: {-fusion_objective(x0, predictor, processor, feature_names):.3f}")

In [ ]:
# Run optimization
try:
    result = optimize.minimize(
        fusion_objective,
        x0,
        args=(predictor, processor, feature_names, True),
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 100}
    )
    
    if result.success:
        optimal_params = result.x
        optimal_q = -result.fun
        
        print("\nOptimization Results:")
        print("=" * 30)
        print(f"Optimization successful: {result.success}")
        print(f"Optimal Q factor: {optimal_q:.3f}")
        print(f"Improvement: {optimal_q - (-fusion_objective(x0, predictor, processor, feature_names)):.3f}")
        
        param_names = ['Magnetic Field (T)', 'Plasma Current (MA)', 'Electron Density (m⁻³)',
                      'Ion Temperature (keV)', 'Electron Temperature (keV)', 
                      'Neutral Beam Power (MW)', 'RF Heating Power (MW)']
        
        print("\nOptimal Parameters:")
        for name, initial, optimal in zip(param_names, x0, optimal_params):
            if 'Density' in name:
                print(f"  {name:<25}: {initial:.2e} → {optimal:.2e}")
            else:
                print(f"  {name:<25}: {initial:.2f} → {optimal:.2f}")
    else:
        print(f"Optimization failed: {result.message}")
        
except Exception as e:
    print(f"Optimization error: {e}")

## 3. Sensitivity Analysis

In [ ]:
# Perform sensitivity analysis - how sensitive is Q factor to parameter changes?
print("Performing sensitivity analysis...")

# Base case parameters
base_case = [5.3, 15.0, 1.0e20, 20.0, 15.0, 50.0, 30.0]
param_names_short = ['B_field', 'I_plasma', 'n_e', 'T_i', 'T_e', 'P_NBI', 'P_RF']

# Calculate base case Q factor
base_q = -fusion_objective(base_case, predictor, processor, feature_names)

# Sensitivity analysis: vary each parameter by ±10%
sensitivity_results = {}
perturbation = 0.1  # 10% variation

for i, param_name in enumerate(param_names_short):
    # Positive perturbation
    params_plus = base_case.copy()
    params_plus[i] *= (1 + perturbation)
    q_plus = -fusion_objective(params_plus, predictor, processor, feature_names)
    
    # Negative perturbation
    params_minus = base_case.copy()
    params_minus[i] *= (1 - perturbation)
    q_minus = -fusion_objective(params_minus, predictor, processor, feature_names)
    
    # Calculate sensitivity (change in Q per % change in parameter)
    sensitivity = (q_plus - q_minus) / (2 * perturbation * base_q) * 100
    
    sensitivity_results[param_name] = {
        'sensitivity': sensitivity,
        'q_plus': q_plus,
        'q_minus': q_minus,
        'delta_q': q_plus - q_minus
    }

print(f"\nBase case Q factor: {base_q:.3f}")
print("\nSensitivity Analysis (% change in Q per % change in parameter):")
print("=" * 70)

# Sort by absolute sensitivity
sorted_sensitivity = sorted(sensitivity_results.items(), 
                          key=lambda x: abs(x[1]['sensitivity']), reverse=True)

for param, results in sorted_sensitivity:
    sens = results['sensitivity']
    print(f"{param:<10}: {sens:+7.2f}% (ΔQ: {results['delta_q']:+.3f})")

In [ ]:
# Visualize sensitivity analysis
params = [param for param, _ in sorted_sensitivity]
sensitivities = [results['sensitivity'] for _, results in sorted_sensitivity]

plt.figure(figsize=(12, 6))
colors = ['red' if s < 0 else 'green' for s in sensitivities]
bars = plt.bar(params, sensitivities, color=colors, alpha=0.7)

plt.xlabel('Parameters')
plt.ylabel('Sensitivity (% change in Q per % change in parameter)')
plt.title('Parameter Sensitivity Analysis')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, sens in zip(bars, sensitivities):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + (0.1 if height > 0 else -0.3),
             f'{sens:.1f}%', ha='center', va='bottom' if height > 0 else 'top')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Interpretation
print("\nInterpretation:")
print("- Positive sensitivity: Increasing parameter increases Q factor")
print("- Negative sensitivity: Increasing parameter decreases Q factor")
print("- Larger absolute values indicate more sensitive parameters")

## 4. Multi-objective Optimization

In [ ]:
# Multi-objective optimization: maximize Q factor while minimizing power consumption
print("Setting up multi-objective optimization...")

def multi_objective_function(params):
    """
    Multi-objective function: maximize Q factor, minimize power consumption.
    """
    try:
        # Calculate Q factor (negative because we minimize)
        q_factor = -fusion_objective(params, predictor, processor, feature_names, maximize=True)
        
        # Calculate total power consumption
        total_power = params[5] + params[6]  # NBI + RF power
        
        # Weighted combination (adjust weights as needed)
        w_q = 1.0      # Weight for Q factor
        w_power = 0.01 # Weight for power (scaled down)
        
        # Objective: minimize (-Q + w_power * Power)
        objective = -w_q * q_factor + w_power * total_power
        
        return objective, q_factor, total_power
        
    except Exception as e:
        return 1000, 0, 1000

# Pareto front analysis - try different weight combinations
pareto_points = []
power_weights = np.linspace(0.001, 0.05, 10)

for w_power in power_weights:
    def weighted_objective(params):
        q_factor = -fusion_objective(params, predictor, processor, feature_names, maximize=True)
        total_power = params[5] + params[6]
        return -1.0 * q_factor + w_power * total_power
    
    try:
        result = optimize.minimize(
            weighted_objective,
            x0,
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 50}
        )
        
        if result.success:
            params = result.x
            q_factor = -fusion_objective(params, predictor, processor, feature_names, maximize=True)
            total_power = params[5] + params[6]
            
            pareto_points.append({
                'q_factor': q_factor,
                'total_power': total_power,
                'weight': w_power,
                'params': params
            })
    except:
        continue

print(f"Generated {len(pareto_points)} Pareto optimal points")

In [ ]:
# Plot Pareto front
if pareto_points:
    q_factors = [p['q_factor'] for p in pareto_points]
    powers = [p['total_power'] for p in pareto_points]
    weights = [p['weight'] for p in pareto_points]
    
    plt.figure(figsize=(12, 8))
    
    # Pareto front
    scatter = plt.scatter(powers, q_factors, c=weights, cmap='viridis', s=100, alpha=0.8)
    plt.colorbar(scatter, label='Power Weight')
    
    # Connect points to show trade-off
    sorted_indices = np.argsort(powers)
    sorted_powers = np.array(powers)[sorted_indices]
    sorted_q = np.array(q_factors)[sorted_indices]
    plt.plot(sorted_powers, sorted_q, 'r--', alpha=0.5, linewidth=1)
    
    plt.xlabel('Total Heating Power (MW)')
    plt.ylabel('Q Factor')
    plt.title('Pareto Front: Q Factor vs Power Consumption')
    plt.grid(True, alpha=0.3)
    
    # Add some annotations
    max_q_idx = np.argmax(q_factors)
    min_power_idx = np.argmin(powers)
    
    plt.annotate(f'Max Q: {q_factors[max_q_idx]:.3f}', 
                xy=(powers[max_q_idx], q_factors[max_q_idx]),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    plt.annotate(f'Min Power: {powers[min_power_idx]:.1f} MW', 
                xy=(powers[min_power_idx], q_factors[min_power_idx]),
                xytext=(10, -20), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # Summary of trade-offs
    print("\nPareto Front Analysis:")
    print("=" * 30)
    print(f"Q factor range: {min(q_factors):.3f} - {max(q_factors):.3f}")
    print(f"Power range: {min(powers):.1f} - {max(powers):.1f} MW")
    print(f"\nTrade-off: {(max(q_factors) - min(q_factors))/(max(powers) - min(powers)):.4f} ΔQ per MW")

## 5. Uncertainty Quantification

In [ ]:
# Uncertainty quantification using Monte Carlo simulation
print("Performing uncertainty quantification...")

# Define parameter uncertainties (standard deviations as % of nominal values)
nominal_params = [5.3, 15.0, 1.0e20, 20.0, 15.0, 50.0, 30.0]
param_uncertainties = [0.05, 0.03, 0.10, 0.08, 0.08, 0.05, 0.05]  # 5%, 3%, 10%, etc.

# Monte Carlo simulation
n_samples = 1000
mc_results = []

np.random.seed(42)
for i in range(n_samples):
    # Sample parameters from normal distributions
    sampled_params = []
    for nominal, uncertainty in zip(nominal_params, param_uncertainties):
        std = nominal * uncertainty
        sample = np.random.normal(nominal, std)
        # Ensure positive values
        sample = max(sample, 0.1 * nominal)
        sampled_params.append(sample)
    
    # Calculate Q factor for sampled parameters
    try:
        q_factor = -fusion_objective(sampled_params, predictor, processor, feature_names, maximize=True)
        mc_results.append({
            'q_factor': q_factor,
            'params': sampled_params
        })
    except:
        continue

print(f"Completed {len(mc_results)} Monte Carlo samples")

# Analyze uncertainty
q_factors_mc = [r['q_factor'] for r in mc_results]
q_mean = np.mean(q_factors_mc)
q_std = np.std(q_factors_mc)
q_95_ci = np.percentile(q_factors_mc, [2.5, 97.5])

print("\nUncertainty Analysis:")
print("=" * 25)
print(f"Mean Q factor: {q_mean:.3f} ± {q_std:.3f}")
print(f"95% Confidence Interval: [{q_95_ci[0]:.3f}, {q_95_ci[1]:.3f}]")
print(f"Coefficient of Variation: {q_std/q_mean*100:.1f}%")

In [ ]:
# Plot uncertainty distributions
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Q factor distribution
axes[0].hist(q_factors_mc, bins=50, alpha=0.7, edgecolor='black', density=True)
axes[0].axvline(q_mean, color='red', linestyle='--', linewidth=2, label=f'Mean: {q_mean:.3f}')
axes[0].axvline(q_95_ci[0], color='orange', linestyle=':', alpha=0.7, label='95% CI')
axes[0].axvline(q_95_ci[1], color='orange', linestyle=':', alpha=0.7)
axes[0].axvline(1.0, color='green', linestyle='-', alpha=0.7, label='Breakeven')
axes[0].set_xlabel('Q Factor')
axes[0].set_ylabel('Probability Density')
axes[0].set_title('Q Factor Uncertainty Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Probability of success metrics
prob_breakeven = (np.array(q_factors_mc) >= 1.0).mean()
prob_ignition = (np.array(q_factors_mc) >= 5.0).mean()
prob_high_perf = (np.array(q_factors_mc) >= 10.0).mean()

categories = ['Breakeven\n(Q ≥ 1)', 'Ignition\n(Q ≥ 5)', 'High Performance\n(Q ≥ 10)']
probabilities = [prob_breakeven, prob_ignition, prob_high_perf]
colors = ['green', 'orange', 'red']

bars = axes[1].bar(categories, probabilities, color=colors, alpha=0.7)
axes[1].set_ylabel('Probability')
axes[1].set_title('Probability of Achieving Performance Targets')
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

# Add percentage labels
for bar, prob in zip(bars, probabilities):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{prob*100:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\nProbability of Success:")
print(f"  Breakeven (Q ≥ 1.0): {prob_breakeven*100:.1f}%")
print(f"  Ignition (Q ≥ 5.0): {prob_ignition*100:.1f}%")
print(f"  High Performance (Q ≥ 10.0): {prob_high_perf*100:.1f}%")

## 6. Scenario Analysis

In [ ]:
# Define different operational scenarios
scenarios = {
    'Conservative': {
        'description': 'Safe operational parameters',
        'params': [4.5, 12.0, 0.8e20, 15.0, 12.0, 40.0, 25.0]
    },
    'Nominal': {
        'description': 'Standard operational parameters', 
        'params': [5.3, 15.0, 1.0e20, 20.0, 15.0, 50.0, 30.0]
    },
    'Aggressive': {
        'description': 'High-performance parameters',
        'params': [6.5, 20.0, 1.5e20, 30.0, 25.0, 80.0, 50.0]
    },
    'ITER-like': {
        'description': 'ITER-inspired parameters',
        'params': [5.3, 15.0, 1.0e20, 25.0, 20.0, 75.0, 40.0]
    }
}

print("Scenario Analysis:")
print("=" * 40)

scenario_results = {}
for name, scenario in scenarios.items():
    params = scenario['params']
    
    # Calculate Q factor
    q_factor = -fusion_objective(params, predictor, processor, feature_names, maximize=True)
    
    # Calculate power consumption
    total_power = params[5] + params[6]  # NBI + RF
    
    # Calculate power density (rough estimate)
    plasma_volume = 4 * np.pi**2 * 6.2 * (2.0**2)  # Rough ITER-like volume estimate
    power_density = total_power / plasma_volume * 1e6  # MW/m³
    
    scenario_results[name] = {
        'q_factor': q_factor,
        'total_power': total_power,
        'power_density': power_density,
        'category': categorize_q_factor(q_factor),
        'params': params
    }
    
    print(f"\n{name} Scenario ({scenario['description']}):")
    print(f"  Q Factor: {q_factor:.3f} ({categorize_q_factor(q_factor)})")
    print(f"  Total Power: {total_power:.1f} MW")
    print(f"  Power Density: {power_density:.2f} MW/m³")
    
    # Key parameters
    param_names = ['B(T)', 'Ip(MA)', 'ne(m⁻³)', 'Ti(keV)', 'Te(keV)', 'PNBI(MW)', 'PRF(MW)']
    print(f"  Parameters: ", end="")
    for i, (pname, pval) in enumerate(zip(param_names, params)):
        if 'ne' in pname:
            print(f"{pname}={pval:.1e}", end="")
        else:
            print(f"{pname}={pval:.1f}", end="")
        if i < len(param_names) - 1:
            print(", ", end="")
    print()

In [ ]:
# Visualize scenario comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

scenario_names = list(scenario_results.keys())
q_factors = [scenario_results[name]['q_factor'] for name in scenario_names]
powers = [scenario_results[name]['total_power'] for name in scenario_names]
power_densities = [scenario_results[name]['power_density'] for name in scenario_names]

# Q factor comparison
colors = ['blue', 'green', 'red', 'purple']
bars1 = axes[0,0].bar(scenario_names, q_factors, color=colors, alpha=0.7)
axes[0,0].axhline(y=1.0, color='orange', linestyle='--', alpha=0.7, label='Breakeven')
axes[0,0].axhline(y=5.0, color='red', linestyle='--', alpha=0.7, label='Ignition')
axes[0,0].set_ylabel('Q Factor')
axes[0,0].set_title('Q Factor by Scenario')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)
for bar, q in zip(bars1, q_factors):
    height = bar.get_height()
    axes[0,0].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                  f'{q:.2f}', ha='center', va='bottom')

# Power consumption comparison
bars2 = axes[0,1].bar(scenario_names, powers, color=colors, alpha=0.7)
axes[0,1].set_ylabel('Total Heating Power (MW)')
axes[0,1].set_title('Power Consumption by Scenario')
axes[0,1].grid(True, alpha=0.3)
for bar, power in zip(bars2, powers):
    height = bar.get_height()
    axes[0,1].text(bar.get_x() + bar.get_width()/2., height + 1,
                  f'{power:.0f}', ha='center', va='bottom')

# Power efficiency (Q factor per MW)
efficiency = [q/p for q, p in zip(q_factors, powers)]
bars3 = axes[1,0].bar(scenario_names, efficiency, color=colors, alpha=0.7)
axes[1,0].set_ylabel('Power Efficiency (Q/MW)')
axes[1,0].set_title('Power Efficiency by Scenario')
axes[1,0].grid(True, alpha=0.3)
for bar, eff in zip(bars3, efficiency):
    height = bar.get_height()
    axes[1,0].text(bar.get_x() + bar.get_width()/2., height + 0.001,
                  f'{eff:.3f}', ha='center', va='bottom')

# Risk assessment (placeholder - could be based on disruption probability)
risk_scores = [0.1, 0.05, 0.3, 0.08]  # Arbitrary risk scores for demonstration
bars4 = axes[1,1].bar(scenario_names, risk_scores, color=colors, alpha=0.7)
axes[1,1].set_ylabel('Risk Score (Disruption Probability)')
axes[1,1].set_title('Risk Assessment by Scenario')
axes[1,1].grid(True, alpha=0.3)
for bar, risk in zip(bars4, risk_scores):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 0.005,
                  f'{risk:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Recommendations
best_q = max(scenario_results.items(), key=lambda x: x[1]['q_factor'])
best_efficiency = max(scenario_results.items(), key=lambda x: x[1]['q_factor']/x[1]['total_power'])

print("\nRecommendations:")
print("=" * 20)
print(f"Highest Q Factor: {best_q[0]} (Q = {best_q[1]['q_factor']:.3f})")
print(f"Best Efficiency: {best_efficiency[0]} (Q/MW = {best_efficiency[1]['q_factor']/best_efficiency[1]['total_power']:.4f})")

## 7. Summary and Conclusions

In [ ]:
# Generate comprehensive analysis summary
print("ADVANCED FUSION ANALYSIS SUMMARY")
print("=" * 50)

print("\n1. TIME SERIES ANALYSIS:")
print(f"   - Analyzed 10-second fusion shot simulation")
print(f"   - Q factor range: {time_series['q_factor'].min():.3f} - {time_series['q_factor'].max():.3f}")
print(f"   - Time above breakeven: {breakeven_time:.1f}s ({breakeven_time/10*100:.1f}%)")
print(f"   - Disruptions detected: {len(disruptions)}")

if 'optimal_q' in locals():
    print("\n2. PARAMETER OPTIMIZATION:")
    print(f"   - Optimization successful: Q factor improved to {optimal_q:.3f}")
    print(f"   - Improvement over baseline: {optimal_q - base_q:.3f}")

print("\n3. SENSITIVITY ANALYSIS:")
most_sensitive = sorted_sensitivity[0]
print(f"   - Most sensitive parameter: {most_sensitive[0]} ({most_sensitive[1]['sensitivity']:+.1f}%)")
print(f"   - Parameter ranking by sensitivity:")
for i, (param, results) in enumerate(sorted_sensitivity[:3]):
    print(f"     {i+1}. {param}: {results['sensitivity']:+.1f}%")

if pareto_points:
    print("\n4. MULTI-OBJECTIVE OPTIMIZATION:")
    print(f"   - Generated {len(pareto_points)} Pareto optimal points")
    print(f"   - Q factor vs power trade-off quantified")
    print(f"   - Trade-off rate: {(max(q_factors) - min(q_factors))/(max(powers) - min(powers)):.4f} ΔQ per MW")

if mc_results:
    print("\n5. UNCERTAINTY QUANTIFICATION:")
    print(f"   - Monte Carlo analysis with {len(mc_results)} samples")
    print(f"   - Q factor: {q_mean:.3f} ± {q_std:.3f} (95% CI: [{q_95_ci[0]:.3f}, {q_95_ci[1]:.3f}])")
    print(f"   - Probability of breakeven: {prob_breakeven*100:.1f}%")
    print(f"   - Probability of ignition: {prob_ignition*100:.1f}%")

print("\n6. SCENARIO ANALYSIS:")
for name, results in scenario_results.items():
    print(f"   - {name}: Q = {results['q_factor']:.3f}, Power = {results['total_power']:.1f} MW")

print("\n7. KEY FINDINGS:")
print("   - Parameter optimization can significantly improve Q factor")
print("   - Sensitivity analysis identifies critical control parameters")
print("   - Trade-offs exist between performance and power consumption")
print("   - Uncertainty quantification provides realistic performance expectations")
print("   - Different operational scenarios have distinct risk/reward profiles")

print("\n8. RECOMMENDATIONS:")
print("   - Focus control on most sensitive parameters for maximum impact")
print("   - Consider multi-objective optimization for practical operations")
print("   - Account for uncertainties in operational planning")
print("   - Use scenario analysis for risk management")
print("   - Implement real-time monitoring for disruption detection")

print("\n" + "=" * 50)
print("Advanced analysis completed successfully!")